# 迭代器和生成器

## 迭代器

在 Python 中一切皆对象，对象的抽象就是类，而对象的集合就是容器。

所有的容器都是可迭代的（iterable），这里的迭代，和枚举不完全一样。

而可迭代对象，可迭代对象调用 iter() 函数，可以得到一个迭代器（iterator）。迭代器可以通过 next() 函数得到下一个元素，实现遍历。迭代器通过 next 的方法，要么得到这个容器的下一个对象，要么得到一个 StopIteration 的错误。

- 示例：判断一个对象是否可迭代


In [1]:
from collections import Iterable

def is_iterable(param):
    try: 
        iter(param) 
        return True
    except TypeError:
        return False

# 使用内建函数实现
def is_iterable_2(param):
    return isinstance(param, Iterable)

params = [
    1234,
    '1234',
    [1, 2, 3, 4],
    set([1, 2, 3, 4]),
    {1:1, 2:2, 3:3, 4:4},
    (1, 2, 3, 4)
]
    
for param in params:
    print('{} is iterable? {}'.format(param, is_iterable_2(param)))


1234 is iterable? False
1234 is iterable? True
[1, 2, 3, 4] is iterable? True
{1, 2, 3, 4} is iterable? True
{1: 1, 2: 2, 3: 3, 4: 4} is iterable? True
(1, 2, 3, 4) is iterable? True


## 生成器

生成器是一种特殊的迭代器（注意这个逻辑关系反之不成立）。使用生成器，你可以写出来更加清晰的代码；合理使用生成器，可以降低内存占用、优化程序结构、提高程序速度。

In [2]:
import os
import psutil

# 显示当前 python 程序占用的内存大小
def show_memory_info(hint):
    pid = os.getpid()
    p = psutil.Process(pid)

    info = p.memory_full_info()
    memory = info.uss / 1024. / 1024
    print('{} memory used: {} MB'.format(hint, memory))

def test_iterator():
    show_memory_info('initing iterator')
    # 声明一个迭代器
    test_list = [i for i in range(100_000_000)]
    show_memory_info('after iterator initiated')
    print(sum(test_list))
    show_memory_info('after sum called')

def test_generator():
    show_memory_info('initing generator')
    # 声明一个生成器
    test_list = (i for i in range(100_000_000))
    show_memory_info('after generator initiated')
    print(sum(test_list))
    show_memory_info('after sum called')

%time test_iterator()
print('------------')
%time test_generator()

initing iterator memory used: 21.7734375 MB
after iterator initiated memory used: 3499.2578125 MB
4999999950000000
after sum called memory used: 3499.2578125 MB
CPU times: user 4.36 s, sys: 1.26 s, total: 5.62 s
Wall time: 5.63 s
------------
initing generator memory used: 1038.21875 MB
after generator initiated memory used: 1038.21875 MB
4999999950000000
after sum called memory used: 1038.22265625 MB
CPU times: user 4.34 s, sys: 11.4 ms, total: 4.35 s
Wall time: 4.36 s


上面的示例可以看出，生成器较节省内存，同时运行效率较高。

迭代器中每个元素在生成后都会保存到内存中，它们占用了巨量的内存，内存不够的话就会出现 OOM 错误。于是，生成器的概念应运而生，在调用 next() 函数的时候，才会生成下一个变量。

- 声明一个迭代器很简单，例如，`[i for i in range(100000000)]` 就可以生成一个包含一亿元素的列表。

  此处实际上是可迭代对象，使用 iter() 转为迭代器，为方便说明不区分可迭代对象和迭代器
  
- 生成器在 Python 的写法是用小括号括起来，`(i for i in range(100000000))` 就初始化了一个生成器。
- 迭代器是一个有限集合，生成器则可以成为一个无限集。
  我们只管调用 next()，生成器根据运算会自动生成新的元素，然后返回。

下面我们使用生成器证明数学恒等式：

```
(1 + 2 + 3 + ... + n)^2 = 1^3 + 2^3 + 3^3 + ... + n^3
```

In [3]:
# 这是一个无限循环的生成器
def generator(k):
    i = 1
    while True:
        yield i ** k
        i += 1

def get_sum(n):
    gen_1 = generator(1)
    gen_3 = generator(3)
    sum_1, sum_3 = 0, 0
    for i in range(n):
        next_1 = next(gen_1)
        next_3 = next(gen_3)
        print('next_1 = {}, next_3 = {}'.format(next_1, next_3))
        sum_1 += next_1
        sum_3 += next_3
    print('{} == {}'.format(sum_1 ** 2, sum_3))


get_sum(8)


next_1 = 1, next_3 = 1
next_1 = 2, next_3 = 8
next_1 = 3, next_3 = 27
next_1 = 4, next_3 = 64
next_1 = 5, next_3 = 125
next_1 = 6, next_3 = 216
next_1 = 7, next_3 = 343
next_1 = 8, next_3 = 512
1296 == 1296


In [4]:
# find index its value equals target
def index_normal(L, target):
    result = []
    for i, num in enumerate(L):
        if num == target:
            result.append(i)
    return result

%timeit index_normal([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2)

837 ns ± 2.61 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
# find index its value equals target
def index_generator(L, target):
    for i, num in enumerate(L):
        if num == target:
            yield i

%timeit index_generator([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2)

227 ns ± 2.39 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## LeetCode 一道题

给定两个序列，判定第一个是不是第二个的子序列。<https://leetcode.com/problems/is-subsequence/>


In [6]:
def is_subsequence(a, b):
    b = iter(b)
    # all 表示全满足则返回 True，否则返回 False
    return all(i in b for i in a)

print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))
print(is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))

True
False


- 让我们来拆解一下

In [7]:
def in_subsequence(a, b):
    # 这步是关键
    b = iter(b)
    return ((i in b) for i in a)

gen1 = in_subsequence([1, 3, 5], [1, 2, 3, 4, 5])
gen2 = in_subsequence([1, 4, 3], [1, 2, 3, 4, 5])

# for x in gen1:
#     print(x)

# print('------------')
# for x in gen2:
#     print(x)

# assert
print(all(gen1))
print(all(gen2))

True
False


- 分析关键一步 `b = iter(b)` 的必要性

In [8]:
import copy

def is_subsequence(a, b):
    b = iter(b)
    for i in a:
        print('i: {} - b: {}'.format(i, list(copy.deepcopy(b))))
        yield i in b

gen = is_subsequence([1, 4, 3], [1, 2, 3, 4, 5])

list(gen)

i: 1 - b: [1, 2, 3, 4, 5]
i: 4 - b: [2, 3, 4, 5]
i: 3 - b: [5]


[True, True, False]

## 思考题

对于一个有限元素的生成器，如果迭代完成后，继续调用 next() ，会发生什么呢？生成器可以遍历多次吗？

答：生成器迭代完成后，继续调用 next() 会抛出 StopIteration，生成器不可以遍历多次（复位生成器例外）。


In [9]:
def test(n):
    return (i for i in range(n))

x = test(3)
print(type(x))
print(x.__next__())
print(x.__next__())
print(x.__next__())
print(x.__next__())


<class 'generator'>
0
1
2


StopIteration: 